In [1]:
import os
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

output = pipe(
    prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
    negative_prompt="bad quality, worse quality, low resolution",
    num_frames=16,
    guidance_scale=2.0,
    num_inference_steps=6,
    generator=torch.Generator("cpu").manual_seed(0),
)
frames = output.frames[0]
export_to_gif(frames, "animatelcm.gif")

/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
Loading pipeline components...:  50%|█████     | 3/6 [00:19<00:22,  7.62s/it]/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeat

'animatelcm.gif'

In [11]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

from diffusers.utils.import_utils import is_xformers_available

device = "cuda:1"
dtype = torch.float16
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=dtype).to(device)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=dtype).to(device)

if is_xformers_available():
    print("enable xformers memory efficient attention")
    pipe.unet.enable_xformers_memory_efficient_attention()
else:
    print("install xformers to enable memory efficient attention")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()


/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
Loading pipeline components...:  67%|██████▋   | 4/6 [01:09<00:38, 19.11s/it]/home/yiming/anaconda3/envs/dmd2/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeat

install xformers to enable memory efficient attention


In [2]:
# prompts = ["Sunset over the sea.",
#            "A quiet beach at dawn, the waves gently lapping at the shore and the sky painted in pastel hues.",
#            "Time-lapse of a coastal landscape transitioning from sunrise to nightfall.",
#            "A serene underwater scene featuring a sea turtle swimming.",
#            "Yellow and black tropical fish dart through the sea.",
#            "a dynamic interaction between the ocean and a large rock.",
#            "The dynamic movement of tall, wispy grasses swaying in the wind.",
#            "Slow pan upward of blazing oak fire in an indoor fireplace.",
#            "A serene waterfall cascading down moss-covered rocks."]

# for prompt in prompts:
#     output = pipe(prompt=prompt,
#                   negative_prompt="bad quality, worse quality, low resolution",
#                   num_frames=16,
#                   guidance_scale=2.0,
#                   num_inference_steps=6,
#                   generator=torch.Generator("cpu").manual_seed(0),
#                 )
#         # export_to_gif(output.frames[0], "AnimateLCM_{}.gif".format(prompt[:30].replace(" ", "_")))
    
# # output = pipe(
# #     prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
# #     negative_prompt="bad quality, worse quality, low resolution",
# #     num_frames=16,
# #     guidance_scale=2.0,
# #     num_inference_steps=6,
# #     generator=torch.Generator("cpu").manual_seed(0),
# # )
# # frames = output.frames[0]
# # export_to_gif(frames, "animatelcm.gif")

  0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: CUDA error: invalid configuration argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
import os
UCF_yaml = '../assets/UCF101_prompts.yaml'
import yaml

with open(UCF_yaml, 'r') as stream:
    try:
        UCF_prompts = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

print(UCF_prompts)
prompts  =list(UCF_prompts.values())
os.makedirs("UCF101", exist_ok=True)

for prompt in prompts:
    output = pipe(prompt=prompt,
                  negative_prompt="bad quality, worse quality, low resolution",
                  num_frames=16,
                  guidance_scale=2.0,
                  num_inference_steps=6,
                  generator=torch.Generator("cpu").manual_seed(0),
                )
    export_to_gif(output.frames[0], "UCF101/AnimateLCM_{}.gif".format(prompt[:30].replace(" ", "_")))

{'ApplyEyeMakeup': 'A girl applying eye makeup.', 'ApplyLipstick': 'A person applying bright lipstick.', 'Archery': 'A person in sportswear aiming an arrow.', 'BabyCrawling': 'A baby crawling with toys around.', 'BalanceBeam': 'An athlete on a balance beam.', 'BandMarching': 'A band marching with instruments.', 'BaseballPitch': 'A pitcher throwing a baseball.', 'Basketball': 'A player dribbling a basketball.', 'BasketballDunk': 'A player performing a basketball dunk.', 'BenchPress': 'An athlete doing a bench press.', 'Biking': 'A cyclist riding on a trail.', 'Billiards': 'A player aiming in billiards.', 'BlowDryHair': 'A person using a blow dryer.', 'BlowingCandles': 'Someone blowing out birthday candles.', 'BodyWeightSquats': 'A person doing squats.', 'Bowling': 'A bowler releasing a bowling ball.', 'BoxingPunchingBag': 'A boxer punching a heavy bag.', 'BoxingSpeedBag': 'A boxer hitting a speed bag.', 'BreastStroke': 'A swimmer doing breaststroke.', 'BrushingTeeth': 'A person brushing

  0%|          | 0/6 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 23.69 GiB total capacity; 1.88 GiB already allocated; 20.62 MiB free; 1.93 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# output = pipe(
#     prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
#     negative_prompt="bad quality, worse quality, low resolution",
#     num_frames=16,
#     guidance_scale=2.0,
#     num_inference_steps=6,
#     generator=torch.Generator("cpu").manual_seed(0),
# )
# frames = output.frames[0]
# export_to_gif(frames, "animatelcm.gif")